In [1]:
import torch
import numpy as np
import os
import pydicom as dicom
import pandas as pd

In [2]:
TRAIN_ON_GPU = torch.cuda.is_available()

if(TRAIN_ON_GPU):
    print('Training on GPU!')
else:
    print('Only CPU available')

Training on GPU!


In [3]:
seed = 41
batch_size = 20
image_size = 28

np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
metadatafile="D:/Datasets/Lung-PET-CT-Dxr/metadata.csv"

# CSV-Datei als DataFrame laden
df = pd.read_csv(metadatafile)
# Wende die Formatierung auf die gesamte Spalte 'File Location' an
df['File Location'] = df['File Location'].apply(lambda x: x.lstrip('.').replace('//', '/'))
dfpet = df[df['Manufacturer']=="PT"]
dfct = df[df['Manufacturer']!="PT"]
dfct

,Series UID,Collection,3rd Party Analysis,Data Description URI,Subject ID,Study UID,Study Description,Study Date,Series Description,Manufacturer,Modality,SOP Class Name,SOP Class UID,Number of Images,File Size,File Location,Download Timestamp
1.3.6.1.4.1.14519.5.2.1.6655.2359.257508444832901632590301540805,Lung-PET-CT-Dx,NaN,https://doi.org/10.7937/TCIA.2020.NNC20461,Lung_Dx-A0001,1.3.6.1.4.1.14519.5.2.1.6655.2359.165554066086...,Chest,04-04-2007,5mm,Philips,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,64,33,75 MB,/Lung-PET-CT-Dx/Lung_Dx-A0001/04-04-2007-NA-Ch...,2024-07-17T14:29:27.025
1.3.6.1.4.1.14519.5.2.1.6655.2359.213534032021332276911485641315,Lung-PET-CT-Dx,NaN,https://doi.org/10.7937/TCIA.2020.NNC20461,Lung_Dx-A0001,1.3.6.1.4.1.14519.5.2.1.6655.2359.165554066086...,Chest,04-04-2007,5mm,Philips,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,64,33,75 MB,/Lung-PET-CT-Dx/Lung_Dx-A0001/04-04-2007-NA-Ch...,2024-07-17T14:29:28.25
1.3.6.1.4.1.14519.5.2.1.6655.2359.241981550240354690198744362919,Lung-PET-CT-Dx,NaN,https://doi.org/10.7937/TCIA.2020.NNC20461,Lung_Dx-A0002,1.3.6.1.4.1.14519.5.2.1.6655.2359.104855313699...,ThoraxAThoraxRoutine Adult,04-25-2007,ThoraxRoutine 8.0.0 B70f,SIEMENS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,30,15,82 MB,/Lung-PET-CT-Dx/Lung_Dx-A0002/04-25-2007-NA-Th...,2024-07-17T14:29:31.26
1.3.6.1.4.1.14519.5.2.1.6655.2359.144797390935445988841819830728,Lung-PET-CT-Dx,NaN,https://doi.org/10.7937/TCIA.2020.NNC20461,Lung_Dx-A0003,1.3.6.1.4.1.14519.5.2.1.6655.2359.179476503242...,ThoraxAThoraxRoutine Adult,07-07-2006,ThoraxRoutine 10.0 B40f,SIEMENS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,27,14,24 MB,/Lung-PET-CT-Dx/Lung_Dx-A0003/07-07-2006-NA-Th...,2024-07-17T14:29:32.517
1.3.6.1.4.1.14519.5.2.1.6655.2359.291101486042760745473304995444,Lung-PET-CT-Dx,NaN,https://doi.org/10.7937/TCIA.2020.NNC20461,Lung_Dx-A0003,1.3.6.1.4.1.14519.5.2.1.6655.2359.191296879859...,Chest,07-25-2006,5mm,Philips,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,1,527,34 KB,/Lung-PET-CT-Dx/Lung_Dx-A0003/07-25-2006-NA-Ch...,2024-07-17T14:29:34.579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.3.6.1.4.1.14519.5.2.1.6655.2359.119781210085728227733751217914,Lung-PET-CT-Dx,NaN,https://doi.org/10.7937/TCIA.2020.NNC20461,Lung_Dx-G0060,1.3.6.1.4.1.14519.5.2.1.6655.2359.305690637242...,PET01PTheadlung Adult,01-21-2011,Thorax 1.0 B70f,SIEMENS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,411,216,90 MB,/Lung-PET-CT-Dx/Lung_Dx-G0060/01-21-2011-NA-PE...,2024-07-17T16:35:03.356
1.3.6.1.4.1.14519.5.2.1.6655.2359.175757871322358372209202025490,Lung-PET-CT-Dx,NaN,https://doi.org/10.7937/TCIA.2020.NNC20461,Lung_Dx-G0062,1.3.6.1.4.1.14519.5.2.1.6655.2359.717357271023...,PET03CBMWholebodyFirstHead Adult,01-26-2011,CT WB 1.0 B30f,SIEMENS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,233,122,97 MB,/Lung-PET-CT-Dx/Lung_Dx-G0062/01-26-2011-NA-PE...,2024-07-17T16:35:16.707
1.3.6.1.4.1.14519.5.2.1.6655.2359.225617357279242405725394357433,Lung-PET-CT-Dx,NaN,https://doi.org/10.7937/TCIA.2020.NNC20461,Lung_Dx-G0062,1.3.6.1.4.1.14519.5.2.1.6655.2359.717357271023...,PET03CBMWholebodyFirstHead Adult,01-26-2011,ALPHA Range2,SIEMENS,CT,Secondary Capture Image Storage,1.2.840.10008.5.1.4.1.1.7,180,142,01 MB,/Lung-PET-CT-Dx/Lung_Dx-G0062/01-26-2011-NA-PE...,2024-07-17T16:35:31.285
1.3.6.1.4.1.14519.5.2.1.6655.2359.333321236537063355599236377825,Lung-PET-CT-Dx,NaN,https://doi.org/10.7937/TCIA.2020.NNC20461,Lung_Dx-G0062,1.3.6.1.4.1.14519.5.2.1.6655.2359.717357271023...,PET03CBMWholebodyFirstHead Adult,01-26-2011,Thorax 1.0 B70f,SIEMENS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,370,195,27 MB,/Lung-PET-CT-Dx/Lung_Dx-G0062/01-26-2011-NA-PE...,2024-07-17T16:35:35.306


In [5]:
root="D:/Datasets/Lung-PET-CT-Dxr"


In [6]:
# Initialisiere das Dictionary, um die Ergebnisse zu speichern
dicom_dict = {}

# Funktion, um die UID einer DICOM-Datei zu extrahieren
def get_dicom_uid(file_path):
    try:
        dicom_data = dicom.dcmread(file_path)
        return dicom_data.SOPInstanceUID  # Gibt die UID zurück
    except Exception as e:
        print(f"Fehler beim Lesen der DICOM-Datei {file_path}: {e}")
        return None


D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0164/04-12-2010-NA-PET02WholebodyOnly Adult-91796/3.000000-PET WB Corrected-16485


In [35]:

# Iteriere durch jede Zeile der 'File Location'-Spalte
for file in dfct['File Location']:

    
    # Erstelle den vollständigen Pfad
    full_path = root + file
    
    
    # Prüfe, ob der Ordner existiert
    if os.path.isdir(full_path):
        print('Folder', full_path,'existiert' )
        # Liste alle DICOM-Dateien im Verzeichnis auf
        dicom_files = [f for f in os.listdir(full_path) if f.endswith('.dcm')]
        
        for dicom_file in dicom_files:
            dicom_file_path = os.path.join(full_path, dicom_file.replace("\\", "/"))
            
            # Holen der UID der DICOM-Datei
            dicom_uid = get_dicom_uid(dicom_file_path)
            
            if dicom_uid:
                # Speichern des Dateipfads und der UID im Dictionary
                dicom_dict[dicom_uid] = dicom_file_path.replace("\\", "/")
    else:
        print(f"Ordner {full_path} existiert nicht.")

# Ausgabe des resultierenden Dictionaries
# Umwandeln des Dictionaries in ein Pandas DataFrame
df = pd.DataFrame(list(dicom_dict.items()), columns=['UID', 'File Path'])

# Speichern des DataFrames in einer CSV-Datei
csv_file_path = 'dicom_files.csv'
df.to_csv(csv_file_path, index=False)

print(f"Die Daten wurden in die Datei {csv_file_path} geschrieben.")


Folder D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0001/04-04-2007-NA-Chest-07990/2.000000-5mm-40805 existiert
Folder D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0001/04-04-2007-NA-Chest-07990/3.000000-5mm-41315 existiert
Folder D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0002/04-25-2007-NA-ThoraxAThoraxRoutine Adult-34834/2.000000-ThoraxRoutine  8.0.0  B70f-62919 existiert
Folder D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0003/07-07-2006-NA-ThoraxAThoraxRoutine Adult-24087/3.000000-ThoraxRoutine  10.0  B40f-30728 existiert
Folder D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0003/07-25-2006-NA-Chest-37655/2.000000-5mm-95444 existiert
Folder D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0003/07-07-2006-NA-ThoraxAThoraxRoutine Adult-24087/2.000000-ThoraxRoutine  10.0  B70f-09713 existiert
Folder D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0002/04-25-2007-NA-ThoraxAThoraxRoutine Adult-34834/3.000000-ThoraxRoutine  8.0.0  B40f-10983 exi

In [62]:
dicom_dict

{'1.3.6.1.4.1.14519.5.2.1.6655.2359.911257607240619696148712587012': 'D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0001/04-04-2007-NA-Chest-07990/2.000000-5mm-40805\\1-01.dcm',
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.184801338381895107635381026857': 'D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0001/04-04-2007-NA-Chest-07990/2.000000-5mm-40805\\1-02.dcm',
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.227252232966919521559290115017': 'D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0001/04-04-2007-NA-Chest-07990/2.000000-5mm-40805\\1-03.dcm',
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.305763411933569638615761005863': 'D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0001/04-04-2007-NA-Chest-07990/2.000000-5mm-40805\\1-04.dcm',
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.892649213333065045357131733969': 'D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0001/04-04-2007-NA-Chest-07990/2.000000-5mm-40805\\1-05.dcm',
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.210616673074470338920284866510': 'D:/Dataset

In [17]:
# CSV-Datei als DataFrame laden
df_uids = pd.read_csv('C:/Users/chris/Documents/GitHub/Masterarbeit_Dominik/Christoph/Object_Detection/dicom_files.csv')
df_uids['File Path'].iloc[0]

'D:/Datasets/Lung-PET-CT-Dxr/Lung-PET-CT-Dx/Lung_Dx-A0001/04-04-2007-NA-Chest-07990/2.000000-5mm-40805\\1-01.dcm'

In [33]:
# Der übergeordnete Ordner, der alle Unterordner enthält
filepath_uid = "D:/Datasets/Lung-PET-CT-Dxr/Annotation"

# Initialisiere ein leeres Dictionary
uid_dict = {}

# Iteriere durch alle Ordner und Dateien im Verzeichnis
for folder_name, subfolders, filenames in os.walk(filepath_uid):
    for filename in filenames:
        # Überprüfe, ob die Datei eine XML-Datei ist
        if filename.endswith(".xml"):
            file_path = os.path.join(folder_name, filename)  # Erstelle den vollständigen Dateipfad
            
            # Der Dateiname ist die UID
            uid = os.path.splitext(filename)[0]  # Entferne die Erweiterung '.xml'
            
            # Füge UID und Dateipfad zum Dictionary hinzu
            uid_dict[uid] = file_path.replace("\\", "/")

# Umwandeln des Dictionaries in ein Pandas DataFrame
df = pd.DataFrame(list(uid_dict.items()), columns=['UID_Annotation', 'File Path_Annotation'])

# Speichern des DataFrames in einer CSV-Datei
csv_file_path = 'annotation_files.csv'
df.to_csv(csv_file_path, index=False)





In [34]:
df_annotations = pd.read_csv('C:/Users/chris/Documents/GitHub/Masterarbeit_Dominik/Christoph/Object_Detection/annotation_files.csv')
df_annotations['File Path_Annotation'].iloc[0]

'D:/Datasets/Lung-PET-CT-Dxr/Annotation/A0001/1.3.6.1.4.1.14519.5.2.1.6655.2359.102500633407588554681658808214.xml'

In [27]:
image_directory = "D:/Datasets/Lung-PET-CT-Dxr/manifest-1608669183333/Lung-PET-CT-Dx/Lung_Dx-A0001"
# annotation_directory = 'dataset/annot'
annotation_directory = "C:/Users/chris/Documents/GitHub/Master/Object_Localization/dataset/dataset/annot"

In [28]:
def loadFileInformation(filename):
    # Erstelle ein leeres Dictionary, um die extrahierten DICOM-Informationen zu speichern.
    information = {}
    
    # Lese die DICOM-Datei mit dem angegebenen Dateinamen ein.
    # dicom.dcmread lädt den Inhalt der DICOM-Datei in ein Dataset-Objekt 'ds'.
    # Das Argument `force=True` ermöglicht das Einlesen, auch wenn die Datei 
    # kleinere Inkonsistenzen oder Fehler im Format hat.
    ds = dicom.dcmread(filename, force=True)
    
    # Greife auf die SOPInstanceUID in den DICOM-Metadaten zu.
    # Die SOPInstanceUID ist eine eindeutige Kennung für diese Bildinstanz.
    # Speichere diese UID im Dictionary unter dem Schlüssel 'dicom_num'.
    information['dicom_num'] = ds.SOPInstanceUID
    
    # Gib das Dictionary zurück, das die UID (SOPInstanceUID) enthält.
    return information


In [18]:
def getUID_path(path):
    dict = {}  # Initialisiere ein leeres Wörterbuch, um UIDs und Dateipfade zu speichern.
    list = os.listdir(path)  # Liste alle Unterverzeichnisse im angegebenen 'path' auf.

    for date in list:  # Schleife über jedes Unterverzeichnis (z. B. ein Datum).
        date_path = os.path.join(path, date)  # Erstelle den vollständigen Pfad für das Datum.
        series_list = os.listdir(date_path)  # Liste alle Serien in diesem Datumspfad auf.
        series_list.sort()  # Sortiere die Serienliste, um eine feste Reihenfolge zu haben.

        for series in series_list:  # Schleife über jede Serie im aktuellen Datumspfad.
            series_path = os.path.join(date_path, series)  # Vollständiger Pfad für die Serie.
            dicom_list = os.listdir(series_path)  # Liste alle DICOM-Dateien in der Serie auf.
            dicom_list.sort()  # Sortiere die DICOM-Dateien, um eine feste Reihenfolge zu haben.

            for dicom in dicom_list:  # Schleife über jede DICOM-Datei in der Serie.
                dicom_path = os.path.join(series_path, dicom)  # Vollständiger Pfad für die DICOM-Datei.
                info = loadFileInformation(dicom_path)  # Lese DICOM-Informationen (einschließlich UID) aus.
                dict[info['dicom_num']] = (dicom_path, dicom)  # Speichere UID als Schlüssel mit Pfad und Dateiname.

    return dict  # Gib das Wörterbuch mit allen gefundenen UIDs und zugehörigen Dateipfaden zurück.


In [29]:
import os

def getUID_path(path):
    dict = {}  # Initialisiere ein leeres Wörterbuch, um UIDs und Dateipfade zu speichern.

    # Verwende os.walk, um alle Verzeichnisse und Dateien ab 'path' rekursiv zu durchsuchen.
    for root, dirs, files in os.walk(path):
        # root: aktuelles Verzeichnis
        # dirs: Liste der Unterverzeichnisse im aktuellen Verzeichnis
        # files: Liste der Dateien im aktuellen Verzeichnis

        # Sortiere die Verzeichnisse und Dateien alphabetisch (optional, für feste Reihenfolge).
        dirs.sort()
        files.sort()

        # Schleife durch jede Datei im aktuellen Verzeichnis
        for dicom in files:
            dicom_path = os.path.join(root, dicom)  # Vollständiger Pfad für die DICOM-Datei
            info = loadFileInformation(dicom_path)  # Lese DICOM-Informationen (einschließlich UID) aus
            dict[info['dicom_num']] = (dicom_path, dicom)  # Speichere UID als Schlüssel mit Pfad und Dateiname

    return dict  # Gib das Wörterbuch mit allen gefundenen UIDs und zugehörigen Dateipfaden zurück.


In [30]:
bilder=getUID_path(image_directory)

In [31]:
bilder

{'1.3.6.1.4.1.14519.5.2.1.6655.2359.911257607240619696148712587012': ('D:/Datasets/Lung-PET-CT-Dxr/manifest-1608669183333/Lung-PET-CT-Dx/Lung_Dx-A0001\\04-04-2007-NA-Chest-07990\\2.000000-5mm-40805\\1-01.dcm',
  '1-01.dcm'),
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.184801338381895107635381026857': ('D:/Datasets/Lung-PET-CT-Dxr/manifest-1608669183333/Lung-PET-CT-Dx/Lung_Dx-A0001\\04-04-2007-NA-Chest-07990\\2.000000-5mm-40805\\1-02.dcm',
  '1-02.dcm'),
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.227252232966919521559290115017': ('D:/Datasets/Lung-PET-CT-Dxr/manifest-1608669183333/Lung-PET-CT-Dx/Lung_Dx-A0001\\04-04-2007-NA-Chest-07990\\2.000000-5mm-40805\\1-03.dcm',
  '1-03.dcm'),
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.305763411933569638615761005863': ('D:/Datasets/Lung-PET-CT-Dxr/manifest-1608669183333/Lung-PET-CT-Dx/Lung_Dx-A0001\\04-04-2007-NA-Chest-07990\\2.000000-5mm-40805\\1-04.dcm',
  '1-04.dcm'),
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.892649213333065045357131733969': ('D:/Datasets/Lung-PET-CT-Dxr/m